## Connect to your GDrive 
In order to train the network on your data, create a directory named `data/`
in the current working directory (cwd) of this notebook (when on colab and connected to gdrive
this would be the `MyDrive/` directory in your gdrive account) and put audio files in it. 

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
# this set the cwd of the notebook
%cd /gdrive/MyDrive 

### Install `mimikit`

In [ ]:
!pip install mimikit==0.2.5

### import and arguments

In [ ]:
import mimikit as mmk

# DATA

# list of files or directories to use as data ("./" is the cwd of the notebook)
sources = ['./data']
# audio sample rate
sr = 22050
# the size of the stft
n_fft = 2048
# hop_length of the
hop_length = n_fft // 4

# NETWORK

# this network takes `shift` fft frames as input and outputs `shift` future frames
shift = 8
# the net contains at least 3 LSTM modules (1 in the Encoder, 2 in the Decoder)
# you can add modules to the Encoder by increasing the next argument
n_lstm = 1
# all LSTM modules have internally the same number of layers :
num_layers = 1
# the dimensionality of the model
model_dim = 1024

# OPTIMIZATION

# how many epochs should we train for
max_epochs = 50
# how many examples are used pro training steps
batch_size = 16
# the learning rate
max_lr = 3e-4
# betas control how fast the network changes its 'learning course'.
# generally, betas should be close but smaller than 1. and be balanced with the batch_size :
# the smaller the batch, the higher the betas 'could be'.
betas = (0.9, 0.93)

# MONITORING

# how often should the network generate during training
every_n_epochs = 2
# how many examples from random prompts should be generated
n_examples = 3
# how many steps (1 step = `shift` fft frames!) should be generated
n_steps = 1000 // shift

print("arguments are ok!")



### create the data

In [ ]:
schema = mmk.Seq2SeqLSTMModel.schema(sr, n_fft=n_fft, hop_length=hop_length)

db_path = 's2s-demo.h5'
print("collecting data...")
db = mmk.Database.create(db_path, sources, schema)
if not len(db.fft.files):
    raise ValueError("Empty db. No audio files were found")
print("successfully created the db.")



### create network and train

In [ ]:
net = mmk.Seq2SeqLSTMModel(
    **mmk.Seq2SeqLSTMModel.dependant_hp(db),
    shift=shift,
    n_lstm=n_lstm,
    num_layers=num_layers,
    model_dim=model_dim,
    batch_size=batch_size,
    max_lr=max_lr,
    div_factor=5,
    betas=betas,

)
print(net.hparams)

dm = mmk.DataModule(net, db, splits=tuple())

cb = mmk.GenerateCallback(every_n_epochs,
                          indices=[None] * n_examples,
                          n_steps=n_steps,
                          play_audios=True,
                          plot_audios=True)

trainer = mmk.get_trainer(root_dir=None,
                          max_epochs=max_epochs,
                          callbacks=[cb],
                          checkpoint_callback=False)
print("here we go!")
trainer.fit(net, datamodule=dm)



----------------------------

<img src="https://ktonal.com/k-circle-bw.png" alt="logo" width="75"/>